In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Process Kaggle Dataset

In [ ]:
miscel = pd.read_csv('miscellaneous.csv') # ques.csv contain https://www.kaggle.com/datasets/radek1/sci-or-not-sci-hypthesis-testing-pack?select=6000_wiki_en_sci_questions_with_excerpts.csv

In [ ]:
miscel.head(2)

,prompt,A,B,C,D,E,answer,wikipedia_excerpt
0,Who was responsible for the reorganisation of ...,Territorial brigades,First line divisions,Training Reserve,Second line divisions,British home army,C,British home army in the First World War: The ...
1,What film earned Rakshit Shetty the Karnataka ...,Rakshit Shetty did not win the Karnataka State...,Nam Areal Ondina,Ulidavaru Kandanthe,The information is not provided in the Wikiped...,Simple Agi Ondh Love Story,C,Rakshit Shetty: Rakshit made his acting debut ...


In [ ]:
def get_correct_answer(row):
    ans = row[str(row['answer']).strip()]
    row['answer'] = ans
    return row

In [ ]:
miscel = miscel.apply(get_correct_answer, axis=1)
miscel.drop(labels=['A', 'B', 'C', 'D', 'E'], axis=1, inplace=True)

In [ ]:
miscel.sample(2)

,prompt,answer,wikipedia_excerpt
5077,What happens to Aidan in the excerpt from Lost...,Aidan grabs a suitcase thrown to him by a man ...,Lost in Blue: Shipwrecked: Aidan gets onto a l...
2272,"How many square miles does White House, Tennes...",11 square miles,"White House, Tennessee: 0 square miles (23 2 k..."


In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
def find_src_noun_chunk(token, doc):
  for chunk in doc.noun_chunks:
    if token.i in range(chunk.start, chunk.end):
      return chunk
  return None

In [ ]:
import random

def remove_information(text, nlp, num_pre=1):
  """Remove parts of the input text, subject subtext to remove is of the form: "adposition + noun_chunk",
  if more instances than num_pre, choose randomly to remove

  Args:
    text: text
    num_pre: number of phrases to remove, default 1

  Returns:
    str: text with parts removed, same text if no target phrase is found
  """
  rm_targets = [] # list(dict: {"text": , "start_id": })
  doc = nlp(text)

  for t_i in range(len(doc)):
    token = doc[t_i]
    if token.pos_ == "ADP":
      for child in token.children:
        if child.pos_ in ['PROPN', 'NOUN', 'PRON']:
          rm_phrase = find_src_noun_chunk(child, doc)
          if not rm_phrase:
            continue

          # add text to list to be removed, and advance token index
          phrase_data = {"text": token.text + ' ' + rm_phrase.text.strip(), "start_id": token.idx}
          rm_targets.append(phrase_data)
          t_i = rm_phrase.end
          break

  rm_ids = random.sample(range(0, len(rm_targets)), min(num_pre, len(rm_targets)))
  rm_ids.sort(reverse=True)

  for rm_idx in rm_ids:
    phrase = rm_targets[rm_idx]
    length = len(phrase['text'])
    text = text[:phrase['start_id']] + " " * length + text[phrase['start_id']+length:]
  return ' '.join(text.split())

In [ ]:
import random

def add_incomplete_question(row):
  num_pre = random.choices([1, 2, 3, 4])[0]
  row['incomplete_prompt'] = remove_information(row['prompt'], nlp, num_pre)
  return row

In [ ]:
new_miscel = miscel.apply(add_incomplete_question, axis=1)

In [ ]:
# additional cleansing

new_miscel = new_miscel[new_miscel['prompt'] != "What are some models that attempt to account for all observations without invoking supplemental non-baryonic matter?"]
new_miscel = new_miscel.reset_index(drop=True)

In [ ]:
new_miscel

,prompt,answer,wikipedia_excerpt,incomplete_prompt
0,Who was responsible for the reorganisation of ...,Training Reserve,British home army in the First World War: The ...,Who was responsible of New Army reserve units ?
1,What film earned Rakshit Shetty the Karnataka ...,Ulidavaru Kandanthe,Rakshit Shetty: Rakshit made his acting debut ...,What film earned Rakshit Shetty the Karnataka ...
2,What is the population of Maklavan?,"Maklavan has a population of 2,170 individuals...","Maklavan: Maklavan (, also Romanized as Mākalā...",What is the population ?
3,What was the stud fee for Empire Maker at Gain...,"$100,000","Empire Maker: In September 2015, it was announ...",What was the stud fee in 2016?
4,What books has Brian J. Bowe published for Ens...,"Books about The Ramones, The Clash, and Judas ...",Brian J. Bowe: He co-edited the 2007 anthology...,What books has Brian J. Bowe published ?
...,...,...,...,...
5984,Who was Edward Kingsley?,"Edward Kingsley was a pioneer in ""art house"" f...",Kingsley-International Pictures: The company w...,Who was Edward Kingsley?
5985,What is the purpose of the box set compilation...,"""Singles 1965–1967"" is a box set compilation o...",Singles 1965–1967: Singles 1965–1967 is a box ...,"What is the purpose ""Singles 1965–1967"" , and ..."
5986,"When was the town of De Kalb, New York organized?",1806,"De Kalb, New York: The town was organized in 1...","When was the town , New York organized?"
5987,What is the reason that Newton's second law ca...,The interaction between particles at the quant...,"Arthur C. Clarke bibliography: k a ""The Haunte...",What is the reason that Newton's second law ca...


In [ ]:
# more cleansing

filtered_ids = []
for r in new_miscel.iterrows():
  if not ("||" in r[1]['wikipedia_excerpt']) and not ("||" in r[1]['prompt']):
    filtered_ids.append(r[0])

new_miscel = new_miscel.iloc[filtered_ids, :]
new_miscel = new_miscel.reset_index(drop=True)

In [ ]:
new_miscel = new_miscel.rename(columns={'prompt': 'question', 'wikipedia_excerpt': 'context', 'incomplete_prompt': 'incomplete_question'})

In [ ]:
def add_source(row):
  row['source_dataset'] = 'kaggle'
  return row
new_miscel = new_miscel.apply(add_source, axis=1)

In [ ]:
new_miscel.head()

,question,answer,context,incomplete_question,source_dataset
0,Who was responsible for the reorganisation of ...,Training Reserve,British home army in the First World War: The ...,Who was responsible of New Army reserve units ?,kaggle
1,What film earned Rakshit Shetty the Karnataka ...,Ulidavaru Kandanthe,Rakshit Shetty: Rakshit made his acting debut ...,What film earned Rakshit Shetty the Karnataka ...,kaggle
2,What is the population of Maklavan?,"Maklavan has a population of 2,170 individuals...","Maklavan: Maklavan (, also Romanized as Mākalā...",What is the population ?,kaggle
3,What was the stud fee for Empire Maker at Gain...,"$100,000","Empire Maker: In September 2015, it was announ...",What was the stud fee in 2016?,kaggle
4,What books has Brian J. Bowe published for Ens...,"Books about The Ramones, The Clash, and Judas ...",Brian J. Bowe: He co-edited the 2007 anthology...,What books has Brian J. Bowe published ?,kaggle


#Process RACE Dataset

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

race = load_dataset("ehovy/race", "all")
race_train, race_val, race_test = pd.DataFrame(race['train']), pd.DataFrame(race['validation']), pd.DataFrame(race['test'])

In [ ]:
race_train.shape, race_val.shape, race_test.shape # 90 - 5 - 5 ratio

((87866, 5), (4887, 5), (4934, 5))

Remove Noise

In [ ]:
race_train, race_val, race_test = race_train.dropna(), race_val.dropna(), race_test.dropna()
race_train, race_val, race_test = race_train.drop(columns='example_id', axis=1), race_val.drop(columns='example_id', axis=1), race_test.drop(columns='example_id', axis=1)

In [ ]:
condition = [len(q.split(" ")) >= 3 for q in race_train['question']]
race_train = race_train[condition]

condition = [len(q.split(" ")) >= 3 for q in race_val['question']]
race_val = race_val[condition]

condition = [len(q.split(" ")) >= 3 for q in race_test['question']]
race_test = race_test[condition]

In [ ]:
race_train = race_train.rename(columns={'article':'context'})
race_val = race_val.rename(columns={'article':'context'})
race_test = race_test.rename(columns={'article':'context'})

Extract Useful Data

In [ ]:
def remove_trailing_blank(text):
  """Remove trailing blank (if found), and replace it with question mark ('?')
  """
  words = [char for char in text]

  b_pos = text.rfind('_')
  if b_pos == -1:
    return text

  if b_pos != 0:
    if words[b_pos - 1] == ' ':
      b_pos -=1

  words = words[:b_pos]
  words.append('?')

  return ''.join(words)

In [ ]:
new_race_train = pd.DataFrame(columns=['context', 'question', 'answer', 'source_dataset'])
new_race_val = pd.DataFrame(columns=['context', 'question', 'answer', 'source_dataset'])
new_race_test = pd.DataFrame(columns=['context', 'question', 'answer', 'source_dataset'])

In [ ]:
for r in race_train.iterrows():
  options = r[1]['options']

  option_ids = ['A', 'B', 'C', 'D']
  answer_id = option_ids.index(r[1]['answer'])

  if r[1]['question'].count('_') >= 2:
    continue
  question = remove_trailing_blank(r[1]['question'])

  # get answer and distractors
  answer = options[answer_id]

  datapoint = {"context": [r[1]['context']], "question": [question], "answer": [answer], 'source_dataset': ['race']}
  new_race_train = pd.concat([new_race_train, pd.DataFrame(datapoint)], axis=0, ignore_index=True)

In [ ]:
for r in race_val.iterrows():
  options = r[1]['options']

  option_ids = ['A', 'B', 'C', 'D']
  answer_id = option_ids.index(r[1]['answer'])

  if r[1]['question'].count('_') >= 2:
    continue
  question = remove_trailing_blank(r[1]['question'])

  # get answer and distractors
  answer = options[answer_id]

  datapoint = {"context": [r[1]['context']], "question": [question], "answer": [answer], 'source_dataset': ['race']}
  new_race_val = pd.concat([new_race_val, pd.DataFrame(datapoint)], axis=0, ignore_index=True)

In [ ]:
for r in race_test.iterrows():
  options = r[1]['options']

  option_ids = ['A', 'B', 'C', 'D']
  answer_id = option_ids.index(r[1]['answer'])

  if r[1]['question'].count('_') >= 2:
    continue
  question = remove_trailing_blank(r[1]['question'])

  # get answer and distractors
  answer = options[answer_id]

  datapoint = {"context": [r[1]['context']], "question": [question], "answer": [answer], 'source_dataset': ['race']}
  new_race_test = pd.concat([new_race_test, pd.DataFrame(datapoint)], axis=0, ignore_index=True)

In [ ]:
# additional cleansing: remove questions requiring summary-related understanding, not poking at specific information
sum_ques_keywords = ['main idea', 'passage', 'article', 'which is true', 'is true', 'is not true', 'is false', 'is correct', 'is not correct', 'statement', 'author', 'writer', 'conclude', 'conclusion', 'infer', 'inferred', 'tell us'] # key words found in questions requiring summarization

filtered_ids = []
for r in new_race_train.iterrows():
  if not any([key in r[1]['question'].lower() for key in sum_ques_keywords]):
    filtered_ids.append(r[0])

new_race_train = new_race_train.iloc[filtered_ids]
new_race_train = new_race_train.reset_index(drop=True)

In [ ]:
filtered_ids = []
for r in new_race_val.iterrows():
  if not any([key in r[1]['question'].lower() for key in sum_ques_keywords]):
    filtered_ids.append(r[0])

new_race_val = new_race_val.iloc[filtered_ids]
new_race_val = new_race_val.reset_index(drop=True)

In [ ]:
filtered_ids = []
for r in new_race_test.iterrows():
  if not any([key in r[1]['question'].lower() for key in sum_ques_keywords]):
    filtered_ids.append(r[0])

new_race_test = new_race_test.iloc[filtered_ids]
new_race_test = new_race_test.reset_index(drop=True)

In [ ]:
import random
def add_incomplete_question_for_race(row):
  num_pre = random.choices([1, 2, 3, 4])[0]
  row['incomplete_question'] = remove_information(row['question'], nlp, num_pre)
  return row

new_race_train = new_race_train.apply(add_incomplete_question_for_race, axis=1)
new_race_val = new_race_val.apply(add_incomplete_question_for_race, axis=1)
new_race_test = new_race_test.apply(add_incomplete_question_for_race, axis=1)

Final Dataset

In [ ]:
final_train = pd.concat([new_race_train, new_miscel], axis=0, ignore_index=True)
final_val = new_race_val
final_test = new_race_test

In [ ]:
# gets only long questions

condition = [len(q.split(" ")) >= 7 for q in final_train['question']]
final_train = final_train[condition]

condition = [len(q.split(" ")) >= 7 for q in final_val['question']]
final_val = final_val[condition]

condition = [len(q.split(" ")) >= 7 for q in final_test['question']]
final_test = final_test[condition]

Extract Only Questions with Long Answers: not used

In [ ]:
def count_words(string):
  return len(str(string).split(' '))

In [ ]:
AVG_LEN_THRES = 8

In [ ]:
filtered_dataset = pd.DataFrame(columns=['context', 'question', 'answer', 'source_dataset', 'incomplete_question'])

In [ ]:
# columns: ['context', 'question', 'answer', 'source_dataset']
for r in final_dataset.iterrows():
  answer = r[1]['answer']
  ans_len = len(str(answer).split(' '))

  if ans_len >= AVG_LEN_THRES:
    datapoint = {"context": [r[1]['context']], "question": [r[1]['question']], "answer": [answer], "incomplete_question": [r[1]['incomplete_question']], 'source_dataset': [r[1]['source_dataset']]}
    filtered_dataset = pd.concat([filtered_dataset, pd.DataFrame(datapoint)], axis=0, ignore_index=True)

In [ ]:
# remove error
filtered_dataset = filtered_dataset[filtered_dataset['question'] != "What are some models that attempt to account for all observations without invoking supplemental non-baryonic matter?"]
filtered_dataset = filtered_dataset.reset_index(drop=True)

In [ ]:
# additional cleansing

filtered_ids = []
for r in filtered_dataset.iterrows():
  if not ("||" in r[1]['context']):
    filtered_ids.append(r[0])

filtered_dataset = filtered_dataset.iloc[filtered_ids, :]
filtered_dataset = filtered_dataset.reset_index(drop=True)

In [ ]:
# more cleaning
import re

def remove_space_before_question_mark(row):
  ques = re.sub(r'\s+', ' ', row['question'])
  row['question'] = ques.replace(' ?', '?')

  ques = re.sub(r'\s+', ' ', row['incomplete_question'])
  row['incomplete_question'] = ques.replace(' ?', '?')
  return row

final_dataset = final_dataset.apply(remove_space_before_question_mark, axis=1)
filtered_dataset = filtered_dataset.apply(remove_space_before_question_mark, axis=1)

In [ ]:
final_dataset.to_csv('final_dataset.csv', index=False)

In [ ]:
filtered_dataset.to_csv("filtered_dataset.csv", index=False)

In [ ]:
# download the file
from google.colab import files

filename = 'final_dataset.csv'
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
filtered_dataset.head(1)

,context,question,answer,source_dataset
0,Many people keep the stamps in a drawer at hom...,The problem of keeping stamps at home is that ?,you do not know how many are left,race


#New Method for Removing Random Spans in Question

Modify "incomplete_question" row from "incomplete_dataset" above

In [ ]:
# additional cleansing
import re

def remove_space_before_question_mark(row):
  ques = re.sub(r'\s+', ' ', row['question'])
  row['question'] = ques.replace(' ?', '?')
  return row

final_dataset = final_dataset.apply(remove_space_before_question_mark, axis=1)

In [ ]:
# gets only long questions

condition = [len(q.split(" ")) >= 7 for q in final_dataset['question']]
final_dataset = final_dataset[condition]

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
import pandas as pd
import numpy as np
from spacy import displacy

pd.set_option('display.max_colwidth', None)

nlp = spacy.load('en_core_web_lg')

In [ ]:
import random
import re

def remove_random_information(nlp, text, max_span_len=5, num_pre=3, min_start=2):
    """Remove randome pieces of information in text

    Args:
        text (str):
        max_span_len (int): max span for each piece of text to remove
        num_pre (int): number of spans to remove
        min_start (int): first token id to can be removed
    Return:
        str: new gapped question
    """
    doc = nlp(text)
    text_ids = [i for i in range(min_start, len(doc)) if doc[i].is_alpha] # ensure only select alphanumeric tokens
    root_inds = random.sample(text_ids, min(len(text_ids),num_pre))
    root_inds.sort()
    span_ids = [] # ids of tokens to be removed

    for i in range(len(root_inds)):
      idx = root_inds[i]
      root_token = doc[idx]
      cur_span_ids = [] # ids of tokens in current span to be removed
      for child in root_token.subtree:
          if i != len(root_inds) - 1 and child.i >= root_inds[i + 1]:
            break
          if len(cur_span_ids) >= max_span_len:
            break
          if child.i < min_start:
            break
          cur_span_ids.append(child)

      # insert start and end ind of span
      if len(cur_span_ids) == 0:
        break
      start_ind = cur_span_ids[0].idx
      end_ind = cur_span_ids[-1].idx + len(cur_span_ids[-1].text)

      span_ids.append((start_ind, end_ind))

    new_chars = []
    for i in range(len(text)):
      flag = True
      for start, end in span_ids:
        if i in range(start, end):
          flag = False

      if flag:
        new_chars.append(text[i])

    new_text = ''.join(new_chars)
    new_text = re.sub(r'\s+', ' ', new_text)
    return new_text

In [ ]:
def preprocess_questions(row):
    row['incomplete_question'] = remove_random_information(nlp, row['question'])
    return row

In [ ]:
final_dataset = final_dataset.apply(preprocess_questions, axis=1)
final_dataset = final_dataset.apply(remove_space_before_question_mark, axis=1)

In [ ]:
final_dataset.shape

(47450, 5)

In [ ]:
final_dataset.to_csv('final_dataset_2.csv', index=False)

In [ ]:
final_dataset.head(15)

,context,question,answer,source_dataset,incomplete_question
0,"After an extra day in space,the crew of Discovery returned to Earth,landing at Edwards Air Force Base in California early on Tuesday morning.The shuttle and its crew spent two weeks in space,most of it aboard the International Space Station.It was the first flight of NASA's spaceshuttle fleet since February 2003,when the shuttle Columbia came apart while reentering Earth's atmosphere.\nNASA officials delayed Discovery's return for one day because of cloudy weather in Florida,where the shuttle was supposed to lanD. On Tuesday morning,mission controllers directed Discovery to a landing site in California,where the skies were clear.The crew members will have to wait until Wednesday to see their families when they all meet together in Houston at the Johnson Space Center.\nDiscovery had a very busy mission in space,compared to past missions.The space shuttle docked with the International Space Station most of the journey,delivering badly needed supplies and repairing damaged parts.The crew spent a lot of time testing new repair techniques on their own shuttle,conducting three different spacewalks(where astronauts exit the space shuttle to do work outside).On the third spacewalk,astronaut Steve Robinson went underneath the shuttle to remove material sticking out from between the spacecraft's protective tiles.\nThere were concerns in the last remaining days of Discovery's mission that a torn heat blanket--another shield against overheating in the space shuttle--may pose a problem to crew members. NASA officials and technicians worked on ways that the crew could fix the problem, but later decided it was not a risk.\nWith the shuttle and its crew safely back on firm ground, NASA is hailing this mission, labeled STS114,as a huge success. ""I hope this shows people that we're coming back,"" NASA spaceflight chief Bill Readdy said after Discovery's successful landing. ""We've got some more work to do. We know what we need to do and we'll do it.",Which is the most probable period of time that Discovery was aboard the International Space Station?,Nine days.,race,Which is most probable period of time that Discovery was aboard International Station?
1,"Anyone who cares about what schools and colleges teach and how their students learn will be interested in the memoir of Ralph W. Tyler, who is one of the most famous men in American education.\nBorn in Chicago in 1902, brought up and schooled in Nebraska, the 19-year-old college graduate Ralph Tyler became _ while teaching as a science teacher in South Dakota and changed his major from medicine to education.\nGraduate work at the University of Chicago found him connected with honorable educators Charles Judd and W. W. Charters, whose ideas of teaching and testing had an effect on his later work. In 1927, he became a teacher of Ohio State University where he further developed a new method of testing.\nTyler became well-known nationality in 1938, when he carried his work with the Eight-Year Study from Ohio State University to the University of Chicago at the invitation of Robert Hutchins.\nTyler was the first director of the Center for Advanced Study in the Behavioral Sciences at Stanford, a position he held for fourteen years. There, he firmly believed that researchers should be free to seek an independent spirit in their work.\nAlthough Tyler officially retired in 1967, he never actually retired. He served on a long list of educational organizations in the United States and abroad. Even in his 80s he traveled across the country to advise teachers and management people on how to set objectives that develop the best teaching and learning within their schools.",Who are most probably interested in Ralph W. Tyler's memoir?,Serious educators.,race,Who are ?
2,"Children in England mustn't work until they are 13. They need to have a work permit to start working.\nThe jobs teenagers can do\nDelivering newspapers\nMany teenagers will get up e

#Process NarrativeQA Dataset

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

nar = load_dataset("deepmind/narrativeqa")

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/32747 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10557 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3461 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
def get_longest_ans(ans_list):
  """
  Return the longest answer out of those provided

  Args:
    ans_list (List(Dict)):
  Returns:
    str: longest ans
  """
  longest_ans = ""
  cur_max_len = 0
  for ans_dict in ans_list:
    if (len(ans_dict['tokens']) > cur_max_len):
      longest_ans = ans_dict['text']
      cur_max_len = len(ans_dict['tokens'])

  return longest_ans

In [ ]:
import pandas as pd

new_nar_train = pd.DataFrame(columns=['context', 'question', 'answer', 'source_dataset'])

for data in nar['train']:
  context = data['document']['summary']['text']
  question = data['question']['text']
  answer = get_longest_ans(data['answers'])
  datapoint = pd.DataFrame({'context': [context], 'question': [question], 'answer': [answer], 'source_dataset': ['narqa']})

  new_nar_train = pd.concat([new_nar_train, pd.DataFrame(datapoint)], axis=0, ignore_index=True)

In [ ]:
new_nar_val = pd.DataFrame(columns=['context', 'question', 'answer', 'source_dataset'])

for data in nar['validation']:
  context = data['document']['summary']['text']
  question = data['question']['text']
  answer = get_longest_ans(data['answers'])
  datapoint = pd.DataFrame({'context': [context], 'question': [question], 'answer': [answer], 'source_dataset': ['narqa']})

  new_nar_val = pd.concat([new_nar_val, pd.DataFrame(datapoint)], axis=0, ignore_index=True)

In [ ]:
new_nar_test = pd.DataFrame(columns=['context', 'question', 'answer', 'source_dataset'])

for data in nar['test']:
  context = data['document']['summary']['text']
  question = data['question']['text']
  answer = get_longest_ans(data['answers'])
  datapoint = pd.DataFrame({'context': [context], 'question': [question], 'answer': [answer], 'source_dataset': ['narqa']})

  new_nar_test = pd.concat([new_nar_test, pd.DataFrame(datapoint)], axis=0, ignore_index=True)

In [ ]:
last_final_train = pd.concat([final_train, new_nar_train], axis=0, ignore_index=True)
last_final_val = pd.concat([final_val, new_nar_val], axis=0, ignore_index=True)
last_final_test = pd.concat([final_test, new_nar_test], axis=0, ignore_index=True)

In [ ]:
# more cleaning
import re

def remove_space_before_question_mark(row):
  ques = re.sub(r'\s+', ' ', row['question'])
  row['question'] = ques.replace(' ?', '?')

  return row

In [ ]:
last_final_train = last_final_train.apply(remove_space_before_question_mark, axis=1)
last_final_val = last_final_val.apply(remove_space_before_question_mark, axis=1)
last_final_test = last_final_test.apply(remove_space_before_question_mark, axis=1)

In [ ]:
last_final_train.source_dataset.value_counts()

source_dataset
race      42536
narqa     32747
kaggle     5764
Name: count, dtype: int64

In [ ]:
last_final_val.source_dataset.value_counts()

source_dataset
narqa    3461
race     2350
Name: count, dtype: int64

In [ ]:
last_final_test.source_dataset.value_counts()

source_dataset
narqa    10557
race      2412
Name: count, dtype: int64

In [ ]:
wh_question_words = [
  "what",
  "who",
  "when",
  "where",
  "why",
  "which",
  "how",
  "whose",
  "that"
]

In [ ]:
# helper
ques = []
for r in last_final_train.sample(500).iterrows():
  if 'how' in r[1]['question'].lower():
    ques.append(r[1]['question'])
    if len(ques) == 50:
      break
ques

In [ ]:
a = [1, 4, 3,4 , 5, 6]
a.index(4)

1

In [ ]:
def extract_key_words(row):
  ques = row['question']
  words = ques.split(' ')
  words = [word.lower().strip() for word in words]
  incomplete_question = ""

  for word in words:
    if word.strip() in wh_question_words:
      incomplete_question = word
      break

  if len(incomplete_question.strip()) == 0: # no predefined keyword found -> get two first and two last words
    incomplete_question = ' '.join(words[:2]) + ' ' + ' '.join(words[-2:])

  incomplete_question = incomplete_question.strip().capitalize()
  incomplete_question = incomplete_question.replace('?', '')
  row['incomplete_question'] = incomplete_question
  return row

In [ ]:
last_final_train = last_final_train.apply(extract_key_words, axis=1)
last_final_val = last_final_val.apply(extract_key_words, axis=1)
last_final_test = last_final_test.apply(extract_key_words, axis=1)

In [ ]:
last_final_train.sample(5)

,context,question,answer,source_dataset,incomplete_question
32133,Trains have long been an important form of tra...,The world's first high speed trains started tr...,200 km/hr,race,The world's traveling at
56049,"The ""William"" of the book's title is Kaiser W...",From which House did Kaiser Wilhilm descend?,The House of Hohenzollern.,narqa,Which
42956,C3 (railcar): The C car is normally at the opp...,How are the C3 cars operated within a consist?,The C3 cars must always face the same directio...,kaggle,How
50296,"The eponymous heroine, Isabel Thorne, is a yo...",What happened to Grimm?,He was captured,narqa,What
28318,"In 2003, Bethany Hamilton,13, lost her left ar...",What can we learn from Anna Sophia Robb's words?,She was absorbed in the story.,race,What


In [ ]:
last_final_train.to_csv('train_set.csv', index=False)
last_final_val.to_csv('validation_set.csv', index=False)
last_final_test.to_csv('test_set.csv', index=False)

# Prep SQuAD Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

squad = load_dataset("rajpurkar/squad")
squad_train, squad_val = pd.DataFrame(squad['train']), pd.DataFrame(squad['validation'])

In [ ]:
# # get long questions only: skipped

# condition = [len(q.split(" ")) >= 3 for q in race_train['question']]
# race_train = race_train[condition]

# condition = [len(q.split(" ")) >= 3 for q in race_val['question']]
# race_val = race_val[condition]

# condition = [len(q.split(" ")) >= 3 for q in race_test['question']]
# race_test = race_test[condition]

In [ ]:
def reformat_squad_ans(row):
  ans_dict = row['answers'] # dict
  answers = ans_dict['text'] # list

  final_ans = ""
  for ans in answers:
    if len(ans) > len(final_ans):
      final_ans = ans

  row['answers'] = final_ans
  return row

In [ ]:
squad_train = squad_train.apply(reformat_squad_ans, axis=1)
squad_val = squad_val.apply(reformat_squad_ans, axis=1)

In [ ]:
squad_train = squad_train.drop(columns=['id', 'title'])
squad_train = squad_train.rename(columns={"answers": 'answer'})

squad_val = squad_val.drop(columns=['id', 'title'])
squad_val = squad_val.rename(columns={"answers": 'answer'})

In [ ]:
# extract keywords

wh_question_words = [
  "what",
  "who",
  "when",
  "where",
  "why",
  "which",
  "how",
  "whose",
  "that"
]

def extract_key_words(row):
  ques = row['question']
  words = ques.split(' ')
  words = [word.lower().strip() for word in words]
  incomplete_question = ""

  for word in words:
    if word.strip() in wh_question_words:
      incomplete_question = word
      break

  if len(incomplete_question.strip()) == 0: # no predefined keyword found -> get two first and two last words
    incomplete_question = ' '.join(words[:2]) + ' ' + ' '.join(words[-2:])

  incomplete_question = incomplete_question.strip().capitalize()
  incomplete_question = incomplete_question.replace('?', '')
  row['incomplete_question'] = incomplete_question
  return row

In [ ]:
squad_train = squad_train.apply(extract_key_words, axis=1)
squad_val = squad_val.apply(extract_key_words, axis=1-\)

In [ ]:
# combine all datasets

train_set = pd.read_csv('train_set.csv')
validation_set = pd.read_csv('validation_set.csv')
test_set = pd.read_csv('test_set.csv')

final_train_set = pd.concat([train_set, squad_train], axis=0, ignore_index=True)
final_validation_set = pd.concat([validation_set, squad_val], axis=0, ignore_index=True)
final_test_set = test_set

In [ ]:
final_train_set.to_csv('final_train_set.csv', index=False)
final_validation_set.to_csv('final_validation_set.csv', index=False)
final_test_set.to_csv('final_test_set.csv', index=False)

In [ ]:
# one part not added: get the last noun chunk of the question using spacy, and add that to the question, so "Why is she playing the guitar?" becomes "Why guitar". Wrote the code somewhere else and too lazy to add it here.